In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [2]:
import os 
import tensorflow as tf
from pathlib import Path

In [3]:
# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:42:00.0, compute capability: 7.5



In [6]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, generate_benchmarks, METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight

In [ ]:
CHEXPERT_BENCHMARKS, CHESTXRAY14_BENCHMARKS = generate_benchmarks ()

In [ ]:
CHEXPERT_BENCHMARKS.keys() 

In [ ]:
CHESTXRAY14_BENCHMARKS.keys()

In [ ]:
#[m.name for m in METRICS]

In [ ]:
#CHEXPERT_COLUMNS

In [8]:
reduced_columns = ['Enlarged Cardiomediastinum',
                    'Cardiomegaly',
                    'Lung Opacity',
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture']
len(reduced_columns)

12

In [12]:
chexpert_bench , _  = generate_benchmarks(path = Path(os.environ.get("CHEXPERT_FULL_PREPROCESSED_DATASET_DIRECTORY")),
                    classes=reduced_columns,
                    batch_sizes = {"b": 32},
                    epoch_sizes = {"e": 12},
                    crop = {"C1": False})

Chestxray_BCE_E12_B32_C1_N12 could not be created
[Errno 2] File data/chexpert/preprocessed-256-crop/meta/data/labels.csv does not exist: 'data/chexpert/preprocessed-256-crop/meta/data/labels.csv'
Chexpert_WBCE_E12_B32_C1_N12 could not be created
[Errno 2] File data/chexpert/preprocessed-256-crop/meta/data/labels.csv does not exist: 'data/chexpert/preprocessed-256-crop/meta/data/labels.csv'
Chestxray_CWBCE_E12_B32_C1_N12 could not be created
[Errno 2] File data/chexpert/preprocessed-256-crop/meta/data/labels.csv does not exist: 'data/chexpert/preprocessed-256-crop/meta/data/labels.csv'


In [38]:
l = list(chexpert_bench.keys())
res = filter(lambda k: "_BCE" in "_"+k and "C1" in k, l)

In [39]:
list(res)

['BCE_E12_B32_C1_N12']

['BCE_E12_B32_C1_N12', 'WBCE_E12_B32_C1_N12', 'CWBCE_E12_B32_C1_N12']

In [40]:
!remote_access/get_tunnels.sh

URLs open for the following services:
[('jupyterlab', 'http://414b26816e9a.ngrok.io'), ('tensorboard', 'http://b71e6637f508.ngrok.io')]


In [ ]:
def simple_architecture_experiment(benchmark, base_model_fn, classes ):
    model = SimpleBaseArchitecture(base_model_fn, len(classes))
    experiment = Experiment(benchmark, model)
    return experiment

In [ ]:
chexpert_resnet_exp = simple_architecture_experiment(CHEXPERT_BENCHMARKS["WBCE_E20_B32"], DenseNet121, CHEXPERT_COLUMNS)


In [ ]:
import random
random.seed()

In [ ]:
chexpert_resnet_exp.run()

In [ ]:
print(chexpert_resnet_exp.evaluation_result["report"])

In [ ]:
chexpert_resnet_exp.model_name

In [ ]:
chestxray14_resnet_exp = simple_architecture_experiment(CHESTXRAY14_BENCHMARKS["WBCE_E20_B32"], DenseNet121, CHESTXRAY14_COLUMNS)

In [ ]:
print(chestxray14_resnet_exp.model_description )


In [ ]:
chestxray14_resnet_exp.run()

In [ ]:
predictions = experiment_inceptionnet_chexpert.model.predict(
            experiment_inceptionnet_chexpert.benchmark.testgen, steps=len(experiment_inceptionnet_chexpert.benchmark.testgen), verbose=1)

In [ ]:
true_labels = experiment_inceptionnet_chexpert.benchmark.testgen.get_encoded_labels()

In [ ]:
# model_guendel_chestxray14 = densenet(classes=len(CHESTXRAY14_COLUMNS))
model_guendel_chexpert = densenet(classes=len(CHEXPERT_COLUMNS))

#experiment_guendel_chestxray14 = Experiment(chestxray14_benchmark, model_guendel_chestxray14)
experiment_guendel_chexpert = Experiment(CHEXPERT_BENCHMARKS["WBCE_E10_B32"], model_guendel_chexpert, model_name="test_WBCE_32")



In [ ]:
#experiment_guendel_chestxray14_result =  experiment_guendel_chestxray14.run()

In [ ]:
experiment_guendel_chexpert_result =  experiment_guendel_chexpert.run()

In [ ]:
experiment_guendel_chexpert.evaluate()

In [ ]:
experiment_guendel_chexpert.save(upload=False)

In [ ]:
print("done")

In [ ]:
CHEXPERT_BENCHMARKS["WBCE_E10_B32"].as_dict()